<a href="https://colab.research.google.com/github/soyeun-data/p-tag/blob/god_father/KBO_Data_%EC%9B%90%ED%95%AB%EC%9D%B8%EC%BD%94%EB%94%A9_tsng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리를 불러옵니다.

In [3]:
# 라이브러리 임포트
import tensorflow as tf
import pandas as pd

데이터를 준비합니다.

1-1. 구글 드라이브 마운트해서 불러오는 방법

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KBO_Data/KBO_all_edited.csv', encoding='CP949')
df.head()

,구장,원정팀,홈팀,원정팀결과
0,목동,LG,넥센,승
1,문학,두산,SK,승
2,대전,롯데,한화,승
3,무등,삼성,기아,승
4,목동,LG,넥센,패


In [ ]:
df.isnull().any()

구장       False
원정팀      False
홈팀       False
원정팀결과     True
dtype: bool

In [ ]:
df = df.dropna(axis=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6759 entries, 0 to 6758
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구장      6759 non-null   object
 1   원정팀     6759 non-null   object
 2   홈팀      6759 non-null   object
 3   원정팀결과   6759 non-null   object
dtypes: object(4)
memory usage: 264.0+ KB


In [ ]:
df = df[df.원정팀결과 != '무'] # 원정팀결과 column에서 무승부만 제거
df.head()

,구장,원정팀,홈팀,원정팀결과
0,목동,LG,넥센,승
1,문학,두산,SK,승
2,대전,롯데,한화,승
3,무등,삼성,기아,승
4,목동,LG,넥센,패


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6627 entries, 0 to 6758
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구장      6627 non-null   object
 1   원정팀     6627 non-null   object
 2   홈팀      6627 non-null   object
 3   원정팀결과   6627 non-null   object
dtypes: object(4)
memory usage: 258.9+ KB


1-2. git에서 바로 불러오는 방법  


In [4]:
# 과거의 데이터를 준비
url = "https://github.com/soyeun-data/p-tag/blob/main/KBO_prediction_data/KBO_2010_season.csv"
df = pd.read_csv(url)
df.head()

ParserError: ignored

1-3. git에서 "KBO_all_season_win_lose.csv" 파일을 불러오는 과정에서 error가 발생.  
ParserError: Error tokenizing data. C error: Expected 1 fields in line 27, saw 367  
error를 해결하기 위해 찾아본 방법
https://zephyrus1111.tistory.com/129

In [5]:
with open('https://github.com/soyeun-data/p-tag/blob/main/KBO_prediction_data/KBO_all_season_win_lose.csv') as f:
    lines = f.readlines()

for line in lines[:27]:
    print(len(line.split(',')))

FileNotFoundError: ignored

1-4. 원핫 인코딩으로 승패를 인코딩합니다.

In [ ]:
# 불필요하다고 생각되는 column 삭제
df10 = df10.drop(labels=['홈팀결과', '비고'], axis=1)
df10.head()

,Date,구장,원정팀,원정팀점수,홈팀점수,홈팀,원정팀결과
0,2010-03-06 13:00,목동,LG,3,2,넥센,승
1,2010-03-06 13:00,문학,두산,5,0,SK,승
2,2010-03-06 13:00,대전,롯데,12,1,한화,승
3,2010-03-06 13:00,무등,삼성,4,1,기아,승
4,2010-03-07 13:00,목동,LG,1,2,넥센,패


In [ ]:
# dataframe에서 특정 column만 원핫인코딩
df = pd.get_dummies(df, columns = ['원정팀결과']) # 무승부가 사라진 걸 확인할 수 있음
df.head()

,구장,원정팀,홈팀,원정팀결과_승,원정팀결과_패
0,목동,LG,넥센,1,0
1,문학,두산,SK,1,0
2,대전,롯데,한화,1,0
3,무등,삼성,기아,1,0
4,목동,LG,넥센,0,1


In [ ]:
# 라벨인코더로 Date 변환하기
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['Date'])
df10['Date'] = le.transform(df10['Date'])

In [ ]:
# 독립변수, 독립변수
input_var10 = df10[['구장', '원정팀', '원정팀점수', '홈팀점수', '홈팀']] # 'Date'를 독립변수에서 빼기로 결정. 전처리 방법을 몰라서...
output_var10 = df10[['원정팀결과_무', '원정팀결과_승', '원정팀결과_패']]
print(input_var10.shape, output_var10.shape)

(594, 5) (594, 3)


In [ ]:
input_var10.head()

,구장,원정팀,원정팀점수,홈팀점수,홈팀
0,목동,LG,3,2,넥센
1,문학,두산,5,0,SK
2,대전,롯데,12,1,한화
3,무등,삼성,4,1,기아
4,목동,LG,1,2,넥센


In [ ]:
output_var10.head()

,원정팀결과_무,원정팀결과_승,원정팀결과_패
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,0,1


In [ ]:
df['구장'].unique()

array(['목동', '문학', '대전', '무등', '대구', '잠실', '사직', '청주', '군산', '마산', '제주',
       '광주', '포항', '상동', '울산', '수원', '고척', '이천(LG)', '창원'], dtype=object)

In [ ]:
# 라벨인코더로 구장 넘버링 변환하기
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['구장'])
df['구장'] = le.transform(df['구장'])

In [ ]:
df['구장'].unique()

array([ 6,  8,  4,  7,  3, 14,  9, 17,  2,  5, 15,  1, 18, 10, 12, 11,  0,
       13, 16])

In [ ]:
df['원정팀'].unique()

array(['LG', '두산', '롯데', '삼성', '기아', '넥센', '한화', 'SK', 'WEST', 'EAST',
       'NC', 'KT', '드림', '나눔', '키움'], dtype=object)

In [ ]:
df['홈팀'].unique()

array(['넥센', 'SK', '한화', '기아', '삼성', '두산', '롯데', 'LG', 'EAST', 'WEST',
       'NC', 'KT', '나눔', '드림', '키움'], dtype=object)

In [ ]:
# array(['목동', '문학', '대전', '무등', '대구', '잠실', '사직', '청주', '군산', '마산']
# 각 구단을 넘버링(라벨링)한다. 
# 그리고 학습을 시키기 위해 dtype을 변환할 예정.

df10 = df10.replace('목동', '1')
df10 = df10.replace('문학', '2')
df10 = df10.replace('대전', '3')
df10 = df10.replace('무등', '4')
df10 = df10.replace('대구', '5')
df10 = df10.replace('잠실', '6')
df10 = df10.replace('사직', '7')
df10 = df10.replace('청주', '8')
df10 = df10.replace('군산', '9')
df10 = df10.replace('마산', '10')

In [ ]:
df = df.replace('LG', '100')
df = df.replace('NC', '101')
df = df.replace('KT', '102')
df = df.replace('SK', '103')
df = df.replace('롯데', '104')
df = df.replace('삼성', '105')
df = df.replace('기아', '106')
df = df.replace('넥센', '107')
df = df.replace('한화', '108')
df = df.replace('두산', '109')
df = df.replace('키움', '110')
df = df.replace('드림', '111')
df = df.replace('나눔', '112')
df = df.replace('WEST', '113')
df = df.replace('EAST', '114')

In [ ]:
print("구장: ", df['구장'].unique())
print("원정팀: ", df['원정팀'].unique())
print("홈팀: ", df['홈팀'].unique())

구장:  [ 6  8  4  7  3 14  9 17  2  5 15  1 18 10 12 11  0 13 16]
원정팀:  ['100' '109' '104' '105' '106' '107' '108' '103' '113' '114' '101' '102'
 '111' '112' '110']
홈팀:  ['107' '103' '108' '106' '105' '109' '104' '100' '114' '113' '101' '102'
 '112' '111' '110']


In [ ]:
df.dtypes

구장          int64
원정팀        object
홈팀         object
원정팀결과_승     uint8
원정팀결과_패     uint8
dtype: object

In [ ]:
#df10['구장'] = df10['구장'].astype('int')
df['원정팀'] = df['원정팀'].astype('int')
df['홈팀'] = df['홈팀'].astype('int')
# df['원정팀결과_무'] = df['원정팀결과_무'].astype('int')
df['원정팀결과_승'] = df['원정팀결과_승'].astype('int')
df['원정팀결과_패'] = df['원정팀결과_패'].astype('int')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6627 entries, 0 to 6758
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   구장       6627 non-null   int64
 1   원정팀      6627 non-null   int64
 2   홈팀       6627 non-null   int64
 3   원정팀결과_승  6627 non-null   int64
 4   원정팀결과_패  6627 non-null   int64
dtypes: int64(5)
memory usage: 310.6 KB


In [ ]:
df = df.replace('-', '')
df = df.replace(':', '')
df = df.replace(' ', '')

In [ ]:
df.tail()

,Date,구장,원정팀,원정팀점수,홈팀점수,홈팀,원정팀결과_무,원정팀결과_승,원정팀결과_패
589,2010-10-13 18:00,5,12,5,6,14,0,0,1
590,2010-10-15 18:00,2,14,5,9,18,0,0,1
591,2010-10-16 14:00,2,14,1,4,18,0,0,1
592,2010-10-18 18:00,5,18,4,2,14,0,1,0
593,2010-10-19 18:00,5,18,4,2,14,0,1,0


2. 모델을 만듭니다.

In [ ]:
# 칼럼 이름 출력
print(df.columns)

Index(['구장', '원정팀', '홈팀', '원정팀결과_무', '원정팀결과_승', '원정팀결과_패'], dtype='object')


In [ ]:
# 독립변수, 독립변수
data = df[['구장', '원정팀', '홈팀']] # 점수 feature를 빼기로 함.
target = df[['원정팀결과_승', '원정팀결과_패']] # 무승부를 빼기로 함.
print(data.shape, target.shape)

(6627, 3) (6627, 2)


In [ ]:
df.describe()

,구장,원정팀,홈팀,원정팀결과_승,원정팀결과_패
count,6627.000000,6627.000000,6627.000000,6627.000000,6627.000000
mean,7.679342,104.781651,104.808058,0.475479,0.524521
std,4.677811,2.915425,2.893727,0.499436,0.499436
min,0.000000,100.000000,100.000000,0.000000,0.000000
25%,4.000000,103.000000,103.000000,0.000000,0.000000
50%,8.000000,105.000000,105.000000,0.000000,1.000000
75%,11.000000,107.000000,107.000000,1.000000,1.000000
max,18.000000,114.000000,114.000000,1.000000,1.000000


In [ ]:
# train과 testt 구분하기

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# train을 이용하여 검증용 sub과 val 구분하기

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(train_input.shape, test_input.shape)
print(sub_input.shape, val_input.shape)

(5301, 3) (1326, 3)
(4240, 3) (1061, 3)


In [ ]:
X = tf.keras.layers.Input(shape=[3])

H = tf.keras.layers.Dense(3)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(2)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('relu')(H)

Y = tf.keras.layers.Dense(2, activation = 'sigmoid')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

2-1. 모델의 구조를 확인합니다.

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 8         
                                                                 
 batch_normalization_1 (Batc  (None, 2)                8         
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
Total params: 22
Trainable params: 18
Non-trainable params: 4
_________________________________________________________________

3. 모델을 학습시킵니다.

sub(in train) data 학습

In [ ]:
model.fit(sub_input, sub_target, epochs=10)

Epoch 1/10
133/133 [==============================] - 2s 3ms/step - loss: 0.7524 - accuracy: 0.4993
Epoch 2/10
133/133 [==============================] - 0s 3ms/step - loss: 0.7088 - accuracy: 0.4877
Epoch 3/10
133/133 [==============================] - 0s 3ms/step - loss: 0.6988 - accuracy: 0.4811
Epoch 4/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6954 - accuracy: 0.4995
Epoch 5/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.5172
Epoch 6/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5156
Epoch 7/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5241
Epoch 8/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6928 - accuracy: 0.5238
Epoch 9/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5224
Epoch 10/10
133/133 [==============================] - 0s 2ms/step - loss: 0.6923 - accuracy: 0.5274

In [ ]:
model.fit(sub_input, sub_target, epochs=100)

Epoch 1/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6924 - accuracy: 0.5262
Epoch 2/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.5262
Epoch 3/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5271
Epoch 4/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5250
Epoch 5/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5278
Epoch 6/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.5269
Epoch 7/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5271
Epoch 8/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5255
Epoch 9/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5274
Epoch 10/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6919 - accura

In [ ]:
model.fit(sub_input, sub_target, epochs=100, verbose=1)

Epoch 1/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5262
Epoch 2/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5267
Epoch 3/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5245
Epoch 4/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5264
Epoch 5/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.5236
Epoch 6/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5255
Epoch 7/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5255
Epoch 8/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5257
Epoch 9/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5257
Epoch 10/100
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accura

In [ ]:
model.fit(sub_input, sub_target, epochs=500, verbose=1)

Epoch 1/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5245
Epoch 2/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5262
Epoch 3/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5281
Epoch 4/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 0.5233
Epoch 5/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5285
Epoch 6/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6918 - accuracy: 0.5241
Epoch 7/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5288
Epoch 8/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5259
Epoch 9/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5238
Epoch 10/500
133/133 [==============================] - 0s 2ms/step - loss: 0.6916 - accura

In [ ]:
print(model.predict(val_input[:10]))

1/1 [==============================] - 0s 126ms/step
[[0.42693466 0.51079035]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]
 [0.4911447  0.5088552 ]]


In [ ]:
print((val_target[:10]))

      원정팀결과_승  원정팀결과_패
2442        1        0
3135        0        1
3057        1        0
301         1        0
5697        1        0
655         0        1
3526        0        1
1772        1        0
1244        1        0
1617        1        0


In [ ]:
df.tail()

,구장,원정팀,홈팀,원정팀결과_승,원정팀결과_패
6754,8,100,103,1,0
6755,1,104,106,1,0
6756,0,105,110,0,1
6757,17,101,108,1,0
6758,16,111,112,1,0


In [ ]:
df11 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KBO_Data/KBO_2011_season.csv')
df11.head()

,Date,구장,원정팀,원정팀점수,홈팀점수,홈팀,원정팀결과,홈팀결과,비고
0,2011-03-12 13:00,대전,LG,6,4,한화,승,패,시범경기
1,2011-03-12 13:00,대구,두산,6,1,삼성,승,패,시범경기
2,2011-03-12 13:00,사직,SK,5,11,롯데,패,승,시범경기
3,2011-03-12 13:00,제주,기아,4,3,넥센,승,패,시범경기
4,2011-03-13 12:00,대구,두산,3,1,삼성,승,패,시범경기


In [ ]:
print(model.evaluate(input_var10, output_var10))

19/19 [==============================] - 0s 4ms/step - loss: 0.7512 - accuracy: 0.5168
[0.7511516809463501, 0.5168350338935852]


In [ ]:
print(model.predict(input_var10[5:15]))
print(output_var10[5:15])

1/1 [==============================] - 0s 22ms/step
[[1.8574484e-03 5.2719051e-01 4.7095206e-01]
 [1.3551192e-03 5.1969528e-01 4.7894958e-01]
 [1.0629552e-03 5.1029736e-01 4.8863971e-01]
 [4.2851265e-03 5.3735256e-01 4.5836225e-01]
 [2.4163837e-03 5.2062899e-01 4.7695461e-01]
 [5.8000680e-04 5.2354121e-01 4.7587878e-01]
 [1.8362687e-03 5.0122607e-01 4.9693769e-01]
 [1.9904093e-03 5.1499146e-01 4.8301813e-01]
 [4.5851786e-03 4.9599025e-01 4.9942458e-01]
 [3.4744054e-04 5.0691295e-01 4.9273959e-01]]
    원정팀결과_무  원정팀결과_승  원정팀결과_패
5         0        0        1
6         0        1        0
7         0        0        1
8         0        0        1
9         0        1        0
10        0        1        0
11        0        1        0
12        0        1        0
13        0        0        1
14        0        1        0
